In [ ]:
# Class: Introduction to Deep Learning
# -------------------------------------------
# Raashil Aadhyanth (U01937668)
# Sujith Gowdru Prabhu Venkatesha (U01883319)
# -------------------------------------------
# Professor : Istavan Barbasi
# Date: 12/14/2024
# -------------------------------------------

In [1]:
from langchain_groq import ChatGroq

In [6]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("When was PACE university founded?")
print(response.content)

Pace University was founded in 1906.


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-46165")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Senior Software Engineer, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySenior Software Engineer, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, 

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)
# We use the code no Preamble so that we dont get intial wordings
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [10]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Software Engineer, ITC',
 'experience': '5-8 years',
 'skills': ['Java',
  'Scala',
  'Node.js',
  'TyperScript',
  'Serverless AWS',
  'ReactJs',
  'Vue',
  'Amazon EC2',
  'GCE',
  'Azure',
  'MySQL',
  'DynamoDB',
  'Elastic Search',
  'CICD',
  'unit testing',
  'static analysis',
  'automated integration tests',
  'Scrum',
  'agile principles',
  'design patterns',
  'distributed processing of large datasets',
  'CI/CD',
  'DevSecOps Engineering',
  'Jenkins',
  'GitHub',
  'SonarQube',
  'Harness',
  'Prisma',
  'New Relic',
  'SignalFx',
  'Splunk',
  'Kafka'],
 'description': 'As a Senior Software Engineer, you will be involved in designing and building secure, highly scalable and innovative services in a continuous delivery model on the Nike cloud platform. This role will partner with leads and architects to deliver web services that are robust, architecturally sound, easily maintainable and highly scalable for various Nike partner experiences and platform.'}

In [11]:
type(json_res)

dict

In [12]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [13]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [20]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/typescript-frontend-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/vue-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/java-portfolio'}],

In [21]:
job

{'role': 'Senior Software Engineer, ITC',
 'experience': '5-8 years',
 'skills': ['Java',
  'Scala',
  'Node.js',
  'TyperScript',
  'Serverless AWS',
  'ReactJs',
  'Vue',
  'Amazon EC2',
  'GCE',
  'Azure',
  'MySQL',
  'DynamoDB',
  'Elastic Search',
  'CICD',
  'unit testing',
  'static analysis',
  'automated integration tests',
  'Scrum',
  'agile principles',
  'design patterns',
  'distributed processing of large datasets',
  'CI/CD',
  'DevSecOps Engineering',
  'Jenkins',
  'GitHub',
  'SonarQube',
  'Harness',
  'Prisma',
  'New Relic',
  'SignalFx',
  'Splunk',
  'Kafka'],
 'description': 'As a Senior Software Engineer, you will be involved in designing and building secure, highly scalable and innovative services in a continuous delivery model on the Nike cloud platform. This role will partner with leads and architects to deliver web services that are robust, architecturally sound, easily maintainable and highly scalable for various Nike partner experiences and platform.'}

In [18]:
job = json_res
job['skills']

['Java',
 'Scala',
 'Node.js',
 'TyperScript',
 'Serverless AWS',
 'ReactJs',
 'Vue',
 'Amazon EC2',
 'GCE',
 'Azure',
 'MySQL',
 'DynamoDB',
 'Elastic Search',
 'CICD',
 'unit testing',
 'static analysis',
 'automated integration tests',
 'Scrum',
 'agile principles',
 'design patterns',
 'distributed processing of large datasets',
 'CI/CD',
 'DevSecOps Engineering',
 'Jenkins',
 'GitHub',
 'SonarQube',
 'Harness',
 'Prisma',
 'New Relic',
 'SignalFx',
 'Splunk',
 'Kafka']

In [23]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Raashil & Sujith, a business development executive's at RaSu Tech. RaSu Tech is an AI Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of RaSu Tech 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase RaSu Tech's portfolio: {link_list}
        Remember you are Raashil & Sujith, BDE at RaSu Tech. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Solutions for Senior Software Engineer, ITC Role at Nike

Dear Hiring Manager,

I came across the Senior Software Engineer, ITC role at Nike and was impressed by the company's commitment to innovation and excellence. As a Business Development Executive at RaSu Tech, I'd like to introduce our company as a potential partner in fulfilling your technology needs.

Our team at RaSu Tech has extensive experience in designing and building secure, highly scalable, and innovative services. We specialize in leveraging cutting-edge technologies such as Java, Scala, Node.js, TypeScript, Serverless AWS, ReactJs, Vue, and more. Our expertise in DevOps, CI/CD, and DevSecOps Engineering ensures seamless integration and deployment of solutions.

We've worked on numerous projects that demonstrate our capabilities in:

* Java: https://example.com/java-portfolio
* ReactJs: https://example.com/react-portfolio
* Vue: https://example.com/vue-portfolio
* DevOps: https://example.com/devops-portf